## 4.4 Simulating a GEDI waveform

In [1]:
from __future__ import print_function, division
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from run_start import *
from hips import hips2img, hips2ani, read_header
from scipy import constants
from IPython.display import HTML

In [2]:
from generate_camera_file import update_existing_camera

In [3]:
# here we can assume our scene units are ~metres, therefore we 
# need to convert our LiDAR pulse into m.

gedi_fwhm = 15
gedi_sigma = gedi_fwhm / (2 * np.sqrt(2 * np.log(2)))

L = 1e-9 * constants.c # speed of light m/ns
pl_ln = 60 * L # pulse length

boomLength = 400000 # height of ISS illumination (and camera) in m
footprint = 22 # diameter at exp(-2) of peak power (86% of energy)

psf_sd = np.exp(1.5) #Gaussian footprint PSF SD is 5.5 but librat needs me to scale it to the footprint...

update_existing_camera('light/pulse.lidar', 'light/large_footprint.lidar.gedi',
                       new_options={'lidar.pulseStart':pl_ln/2.0,
                                    'lidar.pulseForm':'gaussian',
                                    'lidar.pulseSD':gedi_sigma,
                                    'lidar.pulseLength':pl_ln,
                                    'lidar.pulseOPFile':'light/gedi_pulse.dat',
                                    'geometry.boomLength':boomLength,
                                    'geometry.idealArea':footprint*2,
                                    'samplingPattern.form':'gaussian',
                                    'samplingPattern.size':[5000, 5000],
                                    'samplingPattern.sd':[psf_sd, psf_sd],
                                    'samplingPattern.centre':[2500, 2500],
                                    'samplingPattern.OPImage':'light/gediGaussian.hips',
                                    'samplingPattern.threshold':np.exp(-3)})

We will also need to update the camera file, using the logic from the previous example: 

If our camera (<code>geometry.boomLength</code>) is 10 units and the top of the tree is at ~6 units then the minimum return distance is 8 units (<code>lidar.binStart</code>). If we set the <code>lidar.binStep</code> to .1 and the total round distance from illumination to ground to camera is 20 then we'll need to set <code>lidar.nBins</code> to $(20-8)/0.1$ 

In [ ]:
oname='output/gedi_footprint' # geometry.idealArea = footprint
hoc = 6 # height of canopy
diff_camera_toc = boomLength - hoc
binStart = diff_camera_toc * 2
binStep = 0.15
nBins = ((boomLength * 2) - binStart) / binStep + 60

update_existing_camera('camera/pulse.lidar', 'camera/large_footprint.lidar.gedi',oname=oname,
                       new_options={'geometry.idealArea':footprint*2,
                                    'geometry.boomLength':boomLength,
                                    'lidar.binStart':binStart,
                                    'lidar.binStep':binStep,
                                    'lidar.nBins':nBins})                                

In [ ]:
# now we can run the simulation
cmd = 'echo 14 camera/large_footprint.lidar.gedi light/large_footprint.lidar.gedi | \
       start -RATm 1 -RATsensor_wavebands wb/gedi_waveband.dat obj/shrubs.obj'
error = run_start(cmd)   

In [ ]:
# we can now visualise the pulsed waveform
pulse = np.loadtxt('light/gedi_pulse.dat')
bins = np.arange(-5000, 5000)[::int(10000/100)] # lidar.pulseLength sampled lidar.pulseSamples times
plt.plot(bins, pulse)

In [ ]:
# and the total reflectance image
if not error:
    ax = hips2img('{}.hips'.format(oname), stretch=False, order=None)

In [ ]:
# and the animated waveform
%matplotlib notebook
if not error:
    anim = hips2ani('{}.hips'.format(oname), imsave='gedi_waveform.mp4')
    HTML(anim.to_html5_video())

In [ ]:
# We can then look at the results of this by plotting
# reflectance against bin number
%matplotlib inline
refl = np.loadtxt('output/gedi_footprint.dat.direct')
plt.plot(refl[:, 0], refl[:, 1:].mean(axis=1))
#plt.yscale('log') # because ground reflectance is high
plt.xlabel('lidar.nBin')
plt.ylabel('reflectance (log)')

In [ ]:
# We can then view the PSF
hips2img('light/gediGaussian.hips', order=[0])